In [43]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 229 kB in 1s (193 kB/s)
Reading package lists... Done


In [44]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [45]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
home_sales = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")

# Show DataFrame
home_sales.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [46]:
# 2. Create a temporary view of the DataFrame.
home_sales.createOrReplaceTempView('home_sales')


In [47]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query_4b = """
SELECT date_built, ROUND(AVG(price), 2)
FROM home_sales
WHERE bedrooms=4
GROUP BY date_built
"""
spark.sql(query_4b).show()


+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2015|           307908.86|
|      2013|           299999.39|
|      2014|           299073.89|
|      2012|           298233.42|
|      2016|           296050.24|
|      2010|           296800.75|
|      2011|            302141.9|
|      2017|           296576.69|
+----------+--------------------+



In [48]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query_3b3b = """
SELECT date_built, ROUND(AVG(price), 2)
FROM home_sales
WHERE bedrooms=3
AND bathrooms=3
GROUP BY date_built
"""
spark.sql(query_3b3b).show()


+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2015|            288770.3|
|      2013|           295962.27|
|      2014|           290852.27|
|      2012|           293683.19|
|      2016|           290555.07|
|      2010|           292859.62|
|      2011|           291117.47|
|      2017|           292676.79|
+----------+--------------------+



In [49]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

query_3b3b2 = """
SELECT date_built, ROUND(AVG(price), 2)
FROM home_sales
WHERE bedrooms=3
AND bathrooms=3
AND floors=2
AND sqft_living >= 2000
GROUP BY date_built
"""
spark.sql(query_3b3b2).show()


+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2015|           297609.97|
|      2013|           303676.79|
|      2014|           298264.72|
|      2012|           307539.97|
|      2016|            293965.1|
|      2010|           285010.22|
|      2011|           276553.81|
|      2017|           280317.58|
+----------+--------------------+



In [50]:
# 6. What is the "view" rating for the average price of a home,
# rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query_view = """
SELECT view, ROUND(AVG(price), 2)
FROM home_sales
WHERE price >= 350000
GROUP BY view
"""

spark.sql(query_view).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  31|           399856.95|
|  85|          1056336.74|
|  65|           736679.93|
|  53|            755214.8|
|  78|          1080649.37|
|  34|           401419.75|
|  81|          1053472.79|
|  28|           402124.62|
|  76|          1058802.78|
|  26|           401506.97|
|  27|           399537.66|
|  44|           400598.05|
|  12|           401501.32|
|  91|          1137372.73|
|  22|           402022.68|
|  93|          1026006.06|
|  47|            398447.5|
|   1|           401044.25|
|  52|           733780.26|
|  13|           398917.98|
+----+--------------------+
only showing top 20 rows

--- 1.1744475364685059 seconds ---


In [51]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [52]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [53]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

query_view = """
SELECT view, ROUND(AVG(price), 2)
FROM home_sales
WHERE price >= 350000
GROUP BY view
"""

spark.sql(query_view).show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  31|           399856.95|
|  85|          1056336.74|
|  65|           736679.93|
|  53|            755214.8|
|  78|          1080649.37|
|  34|           401419.75|
|  81|          1053472.79|
|  28|           402124.62|
|  76|          1058802.78|
|  26|           401506.97|
|  27|           399537.66|
|  44|           400598.05|
|  12|           401501.32|
|  91|          1137372.73|
|  22|           402022.68|
|  93|          1026006.06|
|  47|            398447.5|
|   1|           401044.25|
|  52|           733780.26|
|  13|           398917.98|
+----+--------------------+
only showing top 20 rows

--- 1.146355390548706 seconds ---


In [64]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales.write.partitionBy('date_built').parquet("home_sales_p")

AnalysisException: ignored

In [65]:
# 11. Read the parquet formatted data.
parquet_data = spark.read.parquet('home_sales_p')

In [66]:
# 12. Create a temporary table for the parquet data.
parquet_data.createOrReplaceTempView('p_home_sales')

In [68]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

query_viewp = """
SELECT view, ROUND(AVG(price), 2)
FROM home_sales
WHERE price >= 350000
GROUP BY view
"""

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.00013256072998046875 seconds ---


In [69]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable('home_sales')

In [74]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

False